In [1]:
from neo4j import GraphDatabase

def get_neo4j_driver() -> GraphDatabase.driver:
    """Establishes and returns a Neo4j session for AuraDB."""
    neo4j_uri = 'neo4j+s://9d1381c2.databases.neo4j.io:7687'
    neo4j_user = 'Shirley'
    neo4j_password = 'Sxl19950312'
    return GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Call the function to get the driver
driver = get_neo4j_driver()

# Open a session
with driver.session() as session:
    # Verify connectivity within the session
    session.run("RETURN 1")

print("Connectivity verified successfully.")


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


# Define a Cypher query to retrieve the required data
query = """
MATCH (n)
RETURN n
"""

# Execute the query and process the results
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Converting Neo4j Data to Pandas DataFrame Format
df = pd.DataFrame(neo4j_data)

#print(df)

Connectivity verified successfully.


In [4]:
import plotly.graph_objects as go
from neo4j import GraphDatabase


query = """
MATCH (ir:IngredientResearch)
RETURN ir.ingredient_name AS IngredientResearchname, ir.alternative_names AS IngredientResearchal
"""


with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]

# Get a list of names and substitutions
names = [record['IngredientResearchname'] for record in neo4j_data]
alternatives = [record['IngredientResearchal'] for record in neo4j_data]

# Cteate chart
fig = go.Figure(data=[go.Table(
    header=dict(values=['IngredientResearchname', 'IngredientResearchal'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[names, alternatives],
               fill_color='lavender',
               align='left'))
])

# Updated Chart Layout
fig.update_layout(title='Ingredient Research Data')

fig.show()



In [5]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (d:DietarySuitability)
RETURN d.diet AS DietarySuitability
"""


with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

# Caculate each DietarySuitability count
diet_counts = df["DietarySuitability"].value_counts().reset_index()
diet_counts.columns = ["DietarySuitability", "Count"]

# use Plotly to create the bar chart
fig = px.bar(diet_counts, x="DietarySuitability", y="Count", title="Number of Dietsuitability with Same Diet")
fig.show()




In [6]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px


query = """
MATCH (d:DietarySuitability)
RETURN d.diet AS Diet, d.suitability AS Suitability
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

# Caculate each dietarysuitability  diet count
diet_counts = df.groupby("Suitability")["Diet"].value_counts().reset_index(name="Count")

# use Plotly to create bar chart
fig = px.bar(diet_counts, x="Diet", y="Count", color="Suitability", title="Diet Distribution for Each DietarySuitability")
fig.show()



In [7]:
from neo4j import GraphDatabase
import pandas as pd
import plotly.express as px



query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, COUNT(n) AS NutritionalInformationCount
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]



df = pd.DataFrame(result, columns=["IngredientResearch", "NutritionalInformationCount"])

fig = px.bar(df, x="IngredientResearch", y="NutritionalInformationCount", title="Number of NutritionInformation per IngredientResearch")
fig.show()



In [8]:
import networkx as nx
import matplotlib.pyplot as plt

query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, n.nutrient AS NutritionalInformation
"""
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)


fig = px.scatter(df, x="IngredientResearch", color="NutritionalInformation", title="NutritionInformation and IngredientResearch")
fig.show()


In [9]:
import networkx as nx
import matplotlib.pyplot as plt

query = """
MATCH (i:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
RETURN i.ingredient_name AS IngredientResearch, n.nutrient AS NutritionalInformation
"""
with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)
# Caculate each dietarysuitability  diet count
diet_counts = df.groupby("IngredientResearch")["NutritionalInformation"].value_counts().reset_index(name="Count")

# use Plotly to create bar chart
fig = px.bar(diet_counts, x="IngredientResearch", y="Count", color="NutritionalInformation", title="IngredientResearch")
fig.show()

In [16]:
import plotly.graph_objs as go
import pandas as pd
from scipy.stats import pearsonr


query = """
MATCH (p:Product)-[:HAS_RECIPE]->(r:Recipe)-[:USES_INGREDIENT]->(i:Ingredient)-[:HAS_RESEARCH]->(a:IngredientResearch)-[:HAS_NUTRITIONAL_INFORMATION]->(n:NutritionalInformation)
OPTIONAL MATCH (p)-[:HAS_RATING]->(h:HealthStarRating)
RETURN p.name AS Product, 
       r.ingredients_orig AS Recipe, 
       i.name AS Ingredient,
       a.ingredient_name AS IngredientResearch, 
       COUNT(n) AS NutritionalinformationCount, 
       h.rating AS HealthStarRating
"""

with driver.session() as session:
    result = session.run(query)
    neo4j_data = [record.data() for record in result]


df = pd.DataFrame(neo4j_data)

#  Remove the null value
df = df.dropna(subset=['NutritionalinformationCount', 'HealthStarRating'])

#  create the scatter
#plt.figure(figsize=(10, 6))
#plt.scatter(df['NutritionalinformationCount'], df['HealthStarRating'], alpha=0.5)
#plt.xlabel('NutritionalinformationCount')
#plt.ylabel('Health Star Rating')
#plt.title('Relationship between Health Star Rating and Nutritionalinformation Count')
#plt.grid(True)
#plt.show()

# Convert "HealthStarRating" column to floating point format
df['HealthStarRating'] = df['HealthStarRating'].astype(float)

# Caculate the correlation
correlation = df['HealthStarRating'].corr(df['NutritionalinformationCount'])
print(f'Correlation between Health Star Rating and Nutritionalinformation Count: {correlation}')

Correlation between Health Star Rating and Nutritionalinformation Count: 0.048391990581925635


In [15]:
import plotly.graph_objs as go
import pandas as pd
from scipy.stats import pearsonr


df = df.dropna(subset=['NutritionalinformationCount', 'HealthStarRating'])

# change "HealthStarRating" column to the float format
df['HealthStarRating'] = df['HealthStarRating'].astype(float)


correlation = df['HealthStarRating'].corr(df['NutritionalinformationCount'])
print(f'Correlation between Health Star Rating and Nutritionalinformation Count: {correlation}')

c#orrelation = df['HealthStarRating'].corr(df['NutritionalinformationCount'])


#print(f'Correlation between Health Star Rating and Nutritionalinformation Count: {correlation:.2f}')

trace = go.Scatter(
    x=df['NutritionalinformationCount'],
    y=df['HealthStarRating'],
    mode='markers',
    text=[f"Product: {product}<br>Recipe: {recipe}<br>Ingredient: {ingredient}<br>IngredientResearch: {research}"
          for product, recipe, ingredient, research in zip(df['Product'], df['Recipe'], df['Ingredient'], df['IngredientResearch'])],
    marker=dict(
        color='rgba(152, 0, 0, 0.8)',
        line=dict(
            color='rgba(0, 0, 0, 0.5)',
            width=1
        )
    )
)

layout = go.Layout(
    title='Relationship between Health Star Rating and Nutritionalinformation Count',
    xaxis=dict(title='NutritionalinformationCount'),
    yaxis=dict(title='Health Star Rating'),
    hovermode='closest'
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()

Correlation between Health Star Rating and Nutritionalinformation Count: 0.048391990581925635


In [ ]:
import plotly.graph_objs as go
import pandas as pd
from scipy.stats import pearsonr


df = df.dropna(subset=['NutritionalinformationCount', 'HealthStarRating'])


df['HealthStarRating'] = df['HealthStarRating'].astype(float)


correlation = df['HealthStarRating'].corr(df['NutritionalinformationCount'])
print(f'Correlation between Health Star Rating and Nutritionalinformation Count: {correlation}')

c#orrelation = df['HealthStarRating'].corr(df['NutritionalinformationCount'])


#print(f'Correlation between Health Star Rating and Nutritionalinformation Count: {correlation:.2f}')

trace = go.Scatter(
    x=df['NutritionalinformationCount'],
    y=df['HealthStarRating'],
    mode='markers',
    text=[f"Product: {product}<br>Recipe: {recipe}<br>Ingredient: {ingredient}<br>IngredientResearch: {research}"
          for product, recipe, ingredient, research in zip(df['Product'], df['Recipe'], df['Ingredient'], df['IngredientResearch'])],
    marker=dict(
        color='rgba(152, 0, 0, 0.8)',
        line=dict(
            color='rgba(0, 0, 0, 0.5)',
            width=1
        )
    )
)

layout = go.Layout(
    title='Relationship between Health Star Rating and Nutritionalinformation Count',
    xaxis=dict(title='NutritionalinformationCount'),
    yaxis=dict(title='Health Star Rating'),
    hovermode='closest'
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()